In [104]:
mask = 0x1
i = 0
comm_size=3
rank=1

while (mask < comm_size):
    dst = rank ^ mask
    
    dst_tree_root = dst >> i
    dst_tree_root <<= i
    
    my_tree_root = rank >> i
    my_tree_root <<= i
    if dst<comm_size:
        print('A', i, dst)
    
    if (dst_tree_root + mask > comm_size):
        nprocs_completed = comm_size - my_tree_root - mask
        print('nprocs_completed: ', nprocs_completed)
        j = mask
        k = 0
        while (j):
            j >>= 1
            k += 1
        k -= 1
        tmp_mask = mask >> 1
        
        while (tmp_mask):
            dst = rank ^ tmp_mask
            tree_root = rank >> k
            tree_root <<= k
            if ((dst > rank) and (rank < tree_root + nprocs_completed) and (dst >= tree_root + nprocs_completed)):
                print('{}- Send to {}'.format(i, dst))
            elif ((dst < rank) and (dst < tree_root + nprocs_completed) and (rank >= tree_root + nprocs_completed)):
                print('{}- Receive {} from {}'.format(i, comm_size - (my_tree_root + mask), dst))
            
            tmp_mask >>= 1
            k -= 1
    
    mask <<= 1
    i+=1

A 0 0
nprocs_completed:  1
1- Receive 1 from 0


In [7]:
n = 3
p = 3

size = n*p


# Inter-Node
for i in range(size):
    left_index = (size + i - p) % size
    right_index = (size + i + p) % size
    
    send_index = i
    recv_index = left_index
    
    for j in range(n-1):
        send_rank = send_index
        recv_rank = recv_index
        
        #print('{} is going to send from {} to {} and receive from {} at {}'.format(i, send_rank, right_index, left_index, recv_rank))
    
        send_index = recv_index
        recv_index = (size + recv_index - p) % size
        
        
# Intra_Node
nodes = [0 for x in range(size)]
for i in range(n):
    for j in range(p):
        nodes[i*p+j] = i
        

for i in range(size):
    right_index = i + 1
    if(right_index >= size):
        right_index -= p
    elif(nodes[i] != nodes[right_index]):
        right_index -= p
        
    left_index = i - 1
    if(left_index < 0):
        left_index += p
    elif(nodes[i] != nodes[right_index]):
        left_index += p
    
    send_index = i
    recv_index = left_index
    
    for j in range(1, p):
        for k in range(n):
            send_rank = (size + send_index - k*p) % size
            recv_rank = (size + recv_index - k*p) % size
            print('{} is going to send from {} to {} and receive from {} at {}'.format(i, send_rank, right_index, left_index, recv_rank))
        send_index = recv_index
        recv_index = recv_index - 1
        if recv_index < 0:
            recv_index += p
        elif(nodes[i] != nodes[recv_index]):
            recv_index += p
            

    

0 is going to send from 0 to 1 and receive from 2 at 2
0 is going to send from 6 to 1 and receive from 2 at 8
0 is going to send from 3 to 1 and receive from 2 at 5
0 is going to send from 2 to 1 and receive from 2 at 1
0 is going to send from 8 to 1 and receive from 2 at 7
0 is going to send from 5 to 1 and receive from 2 at 4
1 is going to send from 1 to 2 and receive from 0 at 0
1 is going to send from 7 to 2 and receive from 0 at 6
1 is going to send from 4 to 2 and receive from 0 at 3
1 is going to send from 0 to 2 and receive from 0 at 2
1 is going to send from 6 to 2 and receive from 0 at 8
1 is going to send from 3 to 2 and receive from 0 at 5
2 is going to send from 2 to 0 and receive from 1 at 1
2 is going to send from 8 to 0 and receive from 1 at 7
2 is going to send from 5 to 0 and receive from 1 at 4
2 is going to send from 1 to 0 and receive from 1 at 0
2 is going to send from 7 to 0 and receive from 1 at 6
2 is going to send from 4 to 0 and receive from 1 at 3
3 is going

# ML RD

In [10]:


n, p = 3, 4

comm_size = n * p


data = [[-1 for x in range(comm_size)] for y in range(comm_size)]
tmp_data = [[-2 for x in range(n)] for y in range(comm_size)]

for i in range(comm_size):
    data[i][i]=i
    tmp_data[i][i//p] = i


curr_cnt = 1
i = 0
mask = 0x1
while (mask < comm_size):
    for rank in range(comm_size):
        my_node = rank//p
        
        dst_node = my_node ^ mask
        dst = rank + (dst_node - my_node) * p


        dst_tree_root = dst_node >> i
        dst_tree_root <<= i



        my_tree_root = my_node >> i
        my_tree_root <<= i

    #    print(dst_node, dst)

        if dst<comm_size:
            #print(rank, dst, dst_node, dst_tree_root, my_tree_root)

            tmp_data[rank][dst_tree_root:min(dst_tree_root+curr_cnt, n)] = tmp_data[dst][dst_tree_root:min(dst_tree_root+curr_cnt, n)]
            #tmp_data[dst][my_tree_root:min(my_tree_root+curr_cnt, comm_size)] = tmp_data[rank][my_tree_root:min(my_tree_root+curr_cnt, comm_size)]
        
            for s in range(min(curr_cnt, n-dst_tree_root)):
                idx = rank + (dst_tree_root+s-my_node) * p
                data[rank][idx] = tmp_data[rank][dst_tree_root+s] 
        if (dst_tree_root + mask > n):
            nprocs_completed = n - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset=my_tree_root + mask
            #print(rank, my_tree_root, mask, offset)
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            tmp_mask = mask >> 1
            
            while (tmp_mask):
                dst_node = my_node ^ tmp_mask
                dst = rank + (dst_node - my_node) * p
        
                tree_root = my_node >> k
                tree_root <<= k
#                 if ((dst_node > my_node) and (my_node < tree_root + nprocs_completed) and (dst_node >= tree_root + nprocs_completed)):
#                     tmp_data[dst][offset:min(offset+curr_cnt, n)] = tmp_data[rank][offset:min(offset+curr_cnt, n)]
#                     for s in range(curr_cnt):
#                         idx = rank + (offset+s-my_node) * p
#                         if idx<comm_size:
#                             data[dst][idx] = tmp_data[dst][offset+s] 
                if ((dst_node < my_node) and (dst_node < tree_root + nprocs_completed) and (my_node >= tree_root + nprocs_completed)):
                    tmp_data[rank][offset:min(offset+curr_cnt, n)] = tmp_data[dst][offset:min(offset+curr_cnt, n)]
                    for s in range(min(curr_cnt, n-offset)):
                        idx = rank + (offset+s-my_node) * p
                        #print(rank, idx)
#                         if idx<comm_size:
                        data[rank][idx] = tmp_data[rank][offset+s] 

                tmp_mask >>= 1
                k -= 1
    curr_cnt *= 2
    mask <<= 1
    i+=1
    
# for d in range(len(data)):
#     print(data[d], d)
print('================= Intra-Node =================')
mask = 0x1
i = 0
curr_cnt = 1

shmem_ranks = [x%p for x in range(comm_size)]

while (mask < p):
    for rank in range(comm_size):
        sh_rank = shmem_ranks[rank]
        sh_dst = sh_rank ^ mask
        my_node = rank//p
        dst = my_node*p + sh_dst

        dst_tree_root = sh_dst >> i
        dst_tree_root <<= i

        
        my_tree_root = sh_rank >> i
        my_tree_root <<= i
        if dst<(my_node+1)*p and dst>=(my_node)*p:
            
            for s in range(n):
                #print(rank, (dst_tree_root+s*p)%comm_size, (my_tree_root+s*p)%comm_size)
                for t in range(min(curr_cnt, p - dst_tree_root)):
                    my_idx = (my_tree_root + s*p) % comm_size +t
                    dst_idx =  (dst_tree_root + s*p) % comm_size + t
#                     if my_idx < comm_size:
#                         print(rank, ' sends to ', dst, ' from ', my_idx)
#                         data[dst][my_idx] = data[rank][my_idx]
#                         if(rank==3 and dst ==5):
#                             print('Yes', data[dst])
#                             print('No', data[rank])
                    if dst_idx < comm_size:
#                         print(rank, ' receives from ', dst, ' at ', dst_idx, dst_tree_root)
                        data[rank][dst_idx] = data[dst][dst_idx]
#                 for tt in range(((my_node + 1) * p - dst_tree_root)):
#                     dst_idx =  (dst_tree_root+s*p)%comm_size + tt
#                     if dst_idx < comm_size:
#                         print(rank, ' receives from ', dst, ' at ', dst_idx)
#                         data[rank][dst_idx] = data[dst][dst_idx]
                    

#         print('----')
#         if (dst_tree_root + mask > p):
#             nprocs_completed = p - my_tree_root - mask
#             #print('nprocs_completed: ', nprocs_completed)
#             offset = my_tree_root + mask
            
#             j = mask
#             k = 0
#             while (j):
#                 j >>= 1
#                 k += 1
#             k -= 1
            
#             tmp_mask = mask >> 1

#             while (tmp_mask):
#                 sh_dst = sh_rank ^ tmp_mask
#                 dst = my_node*p + sh_dst
#                 tree_root = sh_rank >> k
#                 tree_root <<= k
                
# #                 print(rank, dst, sh_rank, sh_dst, tree_root, nprocs_completed) 
# #                 print(rank, ' is here with dst = ', dst) 
#                 if ((sh_dst > sh_rank) and (sh_rank < tree_root + nprocs_completed) and (sh_dst >= tree_root + nprocs_completed)):
# #                     print('S ', rank, ' is here with dst = ', dst) 
#                     for s in range(n):
#                         for t in range(min(curr_cnt, p-(my_tree_root + mask))):
# #                             print(rank, ' Sends to ', dst, ' from ', idx)
#                             idx =  (offset+s*p)%comm_size + t
# #                             if idx < comm_size:
# #                                 data[dst][idx] = data[rank][idx]
                
#                 elif ((sh_dst < sh_rank) and (sh_dst < tree_root + nprocs_completed) and (sh_rank >= tree_root + nprocs_completed)):
# #                     print(rank, ' is here with dst = ', dst) 
#                     for s in range(n):
#                         for t in range(min(curr_cnt, p-(my_tree_root + mask))):
#                             idx =  (offset+s*p)%comm_size + t
#                             if idx < comm_size:
#                                 print(rank, ' recievs from ', dst, ' at ', idx)
#                                 data[rank][idx] = data[dst][idx]
                
#                 tmp_mask >>= 1
#                 k -= 1
    
#     for d in range(len(data)):
#         print(data[d], d)
#     print('-----------------------------------------------')
    mask <<= 1
    i+=1
    curr_cnt *= 2
    
for d in range(len(data)):
    print(data[d], d)

================= Intra-Node =================
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 1
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 3
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 4
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 5
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 6
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 7
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 8
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 10
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 11


In [2]:
n, p = 4, 3
comm_size = n * p

mask = 0x1
i = 0
curr_cnt = 1

shmem_ranks = [x%p for x in range(comm_size)]

while (mask < comm_size):
    for rank in range(comm_size):
        sh_rank = shmem_ranks[rank]
        sh_dst = sh_rank ^ mask
        my_node = rank//p
        dst = my_node*p + sh_dst
        print(sh_rank, sh_dst, rank, dst)
    print('------------------------')
    mask <<= 1
    i+=1

0 1 0 1
1 0 1 0
2 3 2 3
0 1 3 4
1 0 4 3
2 3 5 6
0 1 6 7
1 0 7 6
2 3 8 9
0 1 9 10
1 0 10 9
2 3 11 12
------------------------
0 2 0 2
1 3 1 3
2 0 2 0
0 2 3 5
1 3 4 6
2 0 5 3
0 2 6 8
1 3 7 9
2 0 8 6
0 2 9 11
1 3 10 12
2 0 11 9
------------------------
0 4 0 4
1 5 1 5
2 6 2 6
0 4 3 7
1 5 4 8
2 6 5 9
0 4 6 10
1 5 7 11
2 6 8 12
0 4 9 13
1 5 10 14
2 6 11 15
------------------------
0 8 0 8
1 9 1 9
2 10 2 10
0 8 3 11
1 9 4 12
2 10 5 13
0 8 6 14
1 9 7 15
2 10 8 16
0 8 9 17
1 9 10 18
2 10 11 19
------------------------
